In [69]:
###### Imports ######
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup
import datetime, re, requests, io, time, random, string
from credentials import email, password
print('It works!!')

It works!!


In [70]:
driver = webdriver.Chrome('/Users/richard/downloads/chromedriver')
time.sleep(3)

In [71]:

def sign_in(email=email, password=password):
    driver.get('https://wallmine.com')
    time.sleep(2)
    driver.find_element_by_xpath('/html/body/main/header/div/ul/li[1]/ul/li[3]/a').click() # click sign in button
    time.sleep(1)
    driver.find_element_by_xpath('//*[@id="new_user"]/div[5]/div[1]/div[2]/a').click() # click on sign in w/ password
    if "We're glad you're back!" in driver.page_source:
        print('On sign in page')
        # sign into website
        driver.find_element_by_xpath('//*[@id="user_email"]').send_keys(email)
        driver.find_element_by_xpath('//*[@id="user_password"]').send_keys(password)
        time.sleep(0.1)
        driver.find_element_by_xpath('//*[@id="new_user"]/div[5]/div[2]/div[1]/button').click()
        time.sleep(3)
    if 'Stock market overview' in driver.page_source:
        print('Sign-In Successful')
    else: 
        print('start over app')

In [72]:
# run the sign in function
sign_in(email, password)

On sign in page
Sign-In Successful
